# Assigment 04 

In this Assigment, we are going to download XBRL files and extract some partes of the files.
We will create a funciontion to download the files and save into a especific folder.

The urls from the files are into a xml files, so we need to parse, find the right tags to get the links

In this assigment, insted of saving all files into memory, we will save into disk, extract only the parts we need than save into a dataframe.

In [49]:
#Importing dependencies
import xml.etree.ElementTree as ET
import pandas as pd
from tqdm import tqdm
tqdm.pandas(desc="Processing")

In [2]:
#This function will download the files and save into disk
def download_file(url, path='./'):
    import urllib.request
    import shutil
    
    file_name = path + url.split('/')[-1]

    # Download the file from `url` and save it locally under `file_name`:
    with urllib.request.urlopen(url) as response, open(file_name, 'wb') as out_file:
        shutil.copyfileobj(response, out_file)

In [3]:
#Path where files will saved
path ='./data/raw/xbrl/'

We can get the list of the files in https://www.sec.gov/structureddata/rss-feeds-submitted-filings

In [4]:
url = 'https://www.sec.gov/Archives/edgar/monthly/xbrlrss-2017-08.xml'

In [5]:
download_file(url,path=path)

Now we've downloaded the index files, let's parse and get the urls to download the xbrl files

In [6]:
#Creating variables to store the data.
companies = []
urls = []
file = []

#Open the xml file to parse and find the url for the xbrl files
xmlfile = path + url.split('/')[-1]
tree = ET.parse(xmlfile)
root = tree.getroot()
item = root.findall('channel/item')

#The for loop will parse the xml file to find the link to the xbrl files we need
for entry in item:   
    
    desc = entry.findtext('description')  
    
    if desc == '10-K':
        companyName = ''
        xbrlurl = ''
        for item in entry.iter():

            if item.tag == '{http://www.sec.gov/Archives/edgar}companyName':
                companyName = item.text
                
            if item.tag =='{http://www.sec.gov/Archives/edgar}xbrlFiles':
                for ite  in item:
                    if ite.attrib['{http://www.sec.gov/Archives/edgar}description'] == 'XBRL INSTANCE DOCUMENT':
                        xbrlurl = ite.attrib['{http://www.sec.gov/Archives/edgar}url']
        
        #Some companies dont have the xbrl file we are going to use
        if  xbrlurl != '': 
            companies.append(companyName)  
            urls.append(xbrlurl)
            file.append(xbrlurl.split('/')[-1])

In [7]:
#Creating a Dataframe to easely manipulate the data
data = pd.DataFrame(data ={'Company': companies,
                           'url': urls,
                           'file': file})

In [8]:
#Creating a sample of 100 documents
data = data.sample(100, random_state=42 )

In [9]:
download = data['url'].progress_apply(download_file, path= path)

Processing: 100%|██████████| 100/100 [01:07<00:00,  1.65it/s]


In [10]:
#Lets create a checkpoint for start here next time
data.to_json('./data/10k-xbrl.json')

# Parsing XBRL files

Now that we have all files, we need to find only the tags we need

In [1]:
#importing libraries
import pandas as pd
from tqdm import tqdm
tqdm.pandas(desc="Processing")

#Loading the data
data = pd.read_json('./data/10k-xbrl.json')

#Path where files will saved
path ='./data/raw/xbrl/'

In [2]:
def xbrl_tag(xbrlfile,path= './',tags=[]):
    import xml.etree.ElementTree as ET
    from bs4 import BeautifulSoup
    import unidecode
    import re
    from nltk.corpus import stopwords

    stops = set(stopwords.words("english"))
    
    text = ''
    xbrlfile = path + xbrlfile
    tree = ET.parse(xbrlfile)
    root = tree.getroot()
    
    for item in root.findall('.//'):
        #Only will get the tag we need
        for tag in tags:
            if item.tag.find(tag) >= 0:          
                text = BeautifulSoup(item.text,"lxml-xml",).get_text()
                text = unidecode.unidecode(text)
                text = re.sub('\s', ' ',text) 
                text = re.sub("[^a-zA-Z]", " ", text)
                text = text.lower().split()
                text = [w for w in text if not w in stops]
                text = " ".join(text)
    return text            

## Importing Commitment and Contigencies Tags

In [3]:
tags = ['CommitmentsAndContingenciesDisclosure', 'ContingenciesDisclosure','CommitmentsDisclosure']

In [4]:
data['Comitiment'] = data['file'].progress_apply(xbrl_tag,path=path,tags=tags)

Processing: 100%|██████████| 100/100 [00:09<00:00, 10.49it/s]


In [5]:
# Show how many companies dont used this tags
print(data['Company'][data['Comitiment'].apply(len) == 0].count(), ' companies don\'t use this tag')

26  companies don't use this tag


##  Importing Income Tax Disclosure

In [6]:
tags = ['IncomeTaxDisclosure']
data['Income_Tax'] = data['file'].progress_apply(xbrl_tag,path=path,tags=tags)

Processing: 100%|██████████| 100/100 [00:10<00:00,  9.13it/s]


In [7]:
print(data['Company'][data['Income_Tax'].apply(len) == 0].count(), ' companies don\'t use this tag')

6  companies don't use this tag


In [8]:
#Preview how the text look
data['Comitiment'][3]

'commitments contingencieslegal proceedingssysco engaged various legal proceedings arisen fully adjudicated likelihood loss legal proceedings based definitions within contingency accounting literature ranges remote reasonably possible probable probable reasonably estimable losses accrued based estimates range potential losses associated matters management believe ultimate resolution proceedings either individually aggregate material adverse effect upon consolidated financial position results operations company however final results legal proceedings cannot predicted certainty company failed prevail one legal matters associated realized losses exceed company current estimates range potential losses company consolidated financial position results operations could materially adversely affected future periods commitmentssysco committed aggregate product purchases resale order benefit centralized approach purchasing majority agreements expire within one year however certain agreements terms

In [9]:
#Lets create a checkpoint for start here next time
data.to_json('./data/10k-xbrl.json')

# Dealing with the text

Now we can analize the text

In [1]:
#importing libraries
import pandas as pd
from tqdm import tqdm
tqdm.pandas(desc="Processing")

#Loading the data
data = pd.read_json('./data/10k-xbrl.json')

In [2]:
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

In [4]:
#Negative words
neg_stemmed = []
url_neg = 'https://raw.githubusercontent.com/jeffreybreen/twitter-sentiment-analysis-tutorial-201107/08a269765a6b185d5f3dd522c876043ba9628715/data/opinion-lexicon-English/negative-words.txt'
neg_words = pd.read_fwf(url_neg,encoding='latin-1',skiprows=34,header=None, 
                       names=['negatives'])

for word in neg_words['negatives']:
    neg_stemmed.append(stemmer.stem(word))

neg_stemmed = set(neg_stemmed)

In [5]:
def word_count(data):
    return len(data.split())

In [44]:
def negative_count (data):
    corpus = (" ".join([stemmer.stem(word) \
                                   for word in data.split()])) 
    return len([word for word in corpus.split() if word in neg_stemmed])

In [49]:
#Creating a new collum for the Commitiment word count 
data['Comitiment word count'] = data['Comitiment'].apply(word_count)
#Creating a new collum for the Commitiment Negative word count
data['Comitiment Negative words'] = data['Comitiment'].apply(negative_count)

In [50]:
data.sample(10)

,Comitiment,Company,Income_Tax,file,url,Comitiment word count,Comitiment Negative words
12,commitments contingencies,AEHR TEST SYSTEMS,income taxes,aehr-20170531.xml,http://www.sec.gov/Archives/edgar/data/1040470...,2,0
75,commitments contingencieslegal matters various...,KENNAMETAL INC,income taxesincome loss income taxes consisted...,kmt-20170630.xml,http://www.sec.gov/Archives/edgar/data/55242/0...,119,6
44,note commitments contingencies operating lease...,SANFILIPPO JOHN B & SON INC,note income taxes provision income taxes based...,jbss-20170629.xml,http://www.sec.gov/Archives/edgar/data/880117/...,155,11
0,,INFINITY DISTRIBUTION INC.,note income taxes,cik1646916-20170531.xml,http://www.sec.gov/Archives/edgar/data/1646916...,0,0
77,commitments contingencies legal proceedingsthe...,MAXIM INTEGRATED PRODUCTS INC,income taxespretax income loss follows year en...,mxim-20170624.xml,http://www.sec.gov/Archives/edgar/data/743316/...,328,25
47,commitments contingencieslegal mattersthe comp...,COTY INC.,income taxes loss income operations income tax...,coty-20170630.xml,http://www.sec.gov/Archives/edgar/data/1024305...,224,9
37,commitments contingenciesoperating leasesthe c...,Cardiovascular Systems Inc,income taxesthe components company overall def...,csii-20170630.xml,http://www.sec.gov/Archives/edgar/data/1180145...,630,89
97,,VIEW SYSTEMS INC,income taxes income tax purposes company net o...,vsym-20161231.xml,http://www.sec.gov/Archives/edgar/data/1075857...,0,0
65,june future cash payments required contracts e...,MSG NETWORKS INC.,income taxesincome tax expense attributable co...,msgn-20170630.xml,http://www.sec.gov/Archives/edgar/data/1469372...,52,0
3,commitments contingencieslegal proceedingssysc...,SYSCO CORP,income taxesincome tax provisionsfor financial...,syy-20170701.xml,http://www.sec.gov/Archives/edgar/data/96021/0...,212,11
